In [1]:
import yaml
import numpy as np
import math

In [2]:
def print_grid(A):
    print('\n'.join([''.join(['{:4}'.format(item) for item in row]) 
      for row in A]))

In [3]:
def tree_walk(*args):
    # arguments' parsing
    node,n,x,y,parent_slot,parent_north,x_parent,y_parent,b,l,epsilon,j,sensors = args[:]
    
    # mapping to the rotational sides. Which side comes afterwards
    rot = [0,2,1,3] 
    # counter of number of nodes
    m = n + 1
    # For the core module, node['slot'] is not present.
    #     This is instead of having an if statement
    try:
        # get the slode that the child is connected to the parent
        slot = node['slot']
        # Find the orientation of the parent and which side will the 
        #     child connect to the parent
        orientation = (rot.index(parent_north)+rot.index(parent_slot))%4
        # Find the north node of the child
        node_north = rot[(orientation + rot.index(slot) + 2 )%4]
        # Calculate the polar angle to find the x-y displacement
        angle = -math.pi*(((orientation+1))%4)/2
        # calculate the new x-y position for the child and add to the stack
        x_parent = x_parent + int(math.cos(angle))
        y_parent = y_parent + int(math.sin(angle))
        x = np.append(x, x_parent)
        y = np.append(y, y_parent)
    except:
        # only for the core module
        slot = 0
        parent_orientation = 0
        x = np.append(x, x_parent)
        y = np.append(y, y_parent)
        node_north = 0  
    
    # Sensors check
    if (not(node['type'] == 'Core' or node['type'] == 'FixedBrick')):
        sensors = sensors + 1.0
    
    # we then recurse through all the children of the child.
    #     The leaf nodes of the tree do not have node['children']
    try:
        n_children = len(node['children'])
        
        # branch check
        if (n_children == 4):
            b = b + 1
        
        # limb counter
        l_count = 0
        
        # for all the available children, recurse
        for i in range(0,n_children):
            child = node['children'].keys()[i]
            
            # limbs check
            try:
                temp = len(node['children'][child]['children'])
            except:
                l_count = l_count + 1
                if (l_count == n_children):
                    l = l + 1
            
            # length of limbs check
            try:
                if (len(node['children'][child]['children']) == 1):
                    epsilon = epsilon + 1
            except:
                pass
            
            # Joint check
            try:
                if (node['type'] == 'FixedBrick' or node['type'] == 'Core'):
                    if (node['children'][child]['type'] == 'ActiveHinge'):
                        for i2 in range (0,len(node['children'][child]['children'])):
                            child2 = node['children'][child]['children'].keys()[i2]
                            try:
                                temp = len(node['children'][child]['children'][child2]['children'])
                                if (node['children'][child]['children'][child2]['type'] == 'FixedBrick' 
                                    or node['children'][child]['children'][child2]['type'] == 'Core'):
                                    j = j + 1
                            except:
                                pass
                                
            except Exception as error:
                # print error
                pass
            
            [m,x,y,b,l,epsilon,j,sensors] = tree_walk(node['children'][child], m,x,y,child,node_north,
                                    x_parent, y_parent, b,l,epsilon,j,sensors)
    except:
        pass
    
    return [m,x,y,b,l,epsilon,j,sensors]


x = np.array([])
y = np.array([])
with open("robot_150.yaml", 'r') as stream:
    try:
        file = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        pass

# Branching: counter for number of modules attached on all four faces
b = 0.0
# Limbs: counter for number of modules with only one face attached to another module
l = 0.0
# Length of limbs: counting the number of moduls which 
#     have two of its faces attached to other modules
epsilon = 0.0
# joints: number of effective joints (i.e.: joints which have both of their faces attached)
j = 0.0
# sensors: number of sensor components
sensors = 0.0

args = [file['body'],0.0,x,y,0,0,0,0,b,l,epsilon,j, sensors]
results = tree_walk(*args)

m,x,y,b,l,epsilon,j,sensors = results[:]

print "number of nodes: {}".format(m)
print_grid(np.array([x,y]))

number of nodes: 15.0
 0.0 0.0 0.0 0.0 0.0-1.0-2.0-3.0-3.0 1.0 2.0 3.0 2.0 2.0 3.0
 0.0-1.0 1.0 2.0 3.0 0.0 0.0 0.0-1.0 0.0 0.0 0.0-1.0-2.0-2.0


In [4]:
# Calculating branching
b_max = math.floor((m-2.0)/3.0)
if (m >= 5): 
    branching = (b/b_max) 
else: 
    branching = 0

In [5]:
# Calculating number of limbs
if (m>=6):
    l_max = ((2*math.floor((m-6)/3) + ((m-6))%3) + 4)
else:
    l_max = m-1
if l_max > 0:
    L = l/l_max
else:
    L = 0

In [6]:
# Calculating convergence
e_max = m-2.0
if (m>=3):
    E = (epsilon/e_max)
else:
    E = 0

In [7]:
# Calculating Length of limbs
m_area = (max(x) - min(x) + 1)*(max(y) - min(y) + 1)
C = m/m_area

In [8]:
# Calculating Joints
j_max = math.floor((m-1.0)/2.0)
if (m>=3):
    J = (j/j_max)
else:
    J = 0

In [9]:
# Calculating Proportion
ps = (max(x) - min(x) + 1)
pl = (max(y) - min(y) + 1)
P = min(ps/pl, pl/ps)

In [10]:
# Calculating Symmetry
oh = abs(max(x))
qh = abs(min(x))
ov = abs(max(y))
qv = abs(min(y))
zh = min(oh/qh, qh/oh)
zv = min(ov/qv, qv/ov)
Z = max(zh, zv)

In [11]:
# Calculating Size
m_max = 100.0
S = m/m_max

In [12]:
# Calculating Sensors
n_Sensors = sensors/m

In [13]:
print ("branching: {}".format(branching))
print ("limbs: {}".format(L))
print ("Length of Limbs: {}".format(E))
print ("Coverage: {}".format(C))
print ("Joints: {}".format(J))
print ("Proportion: {}".format(P))
print ("Symmetry: {}".format(Z))
print ("Size: {}".format(S))
print ("Sensors: {}".format(n_Sensors))

branching: 0.25
limbs: 0.3
Length of Limbs: 0.615384615385
Coverage: 0.357142857143
Joints: 0.142857142857
Proportion: 0.857142857143
Symmetry: 1.0
Size: 0.15
Sensors: 0.4
